Imports

In [19]:
import feed_forward
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
torch.cuda.is_available()

True

CUDA check

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
                                ])

In [22]:
trainset = datasets.CIFAR10('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.CIFAR10('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(0.03125*img, label_map[label]) for img, label in trainset if label in [0, 2]]
cifar2_val = [(0.03125*img, label_map[label]) for img, label in valset if label in [0, 2]]
trainloader = torch.utils.data.DataLoader(cifar2, batch_size=512, shuffle=True)
valloader = torch.utils.data.DataLoader(cifar2_val, batch_size=512, shuffle=False)

Initializing the model

In [24]:
model = feed_forward.HypFF(3072, 512, 256, 2)
model

HypFF(
  (fc1): MobLinear(
    in_features=3072, out_features=512, bias=True
    (ball): PoincareBall manifold
  )
  (fc2): MobLinear(
    in_features=512, out_features=256, bias=True
    (ball): PoincareBall manifold
  )
  (fc3): MobLinear(
    in_features=256, out_features=2, bias=True
    (ball): PoincareBall manifold
  )
)

In [25]:
learning_rate = 16e-2
momentum = 0.9

In [26]:
optimizer = geoopt.optim.RiemannianSGD(model.parameters(), 
                                       lr=learning_rate)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
loss_fn = nn.NLLLoss()

TypeError: __init__() got an unexpected keyword argument 'momentum'

In [ ]:
n_epochs = 25
for epoch in range(n_epochs):
    for imgs, labels in trainloader:
        # img, label = img.to(device), label.to(device)
        batch_size = imgs.shape[0]
        out = model(imgs.view(batch_size, -1))
        loss = loss_fn(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.499387
Epoch: 1, Loss: 0.492084
Epoch: 2, Loss: 0.532251
Epoch: 3, Loss: 0.462872
Epoch: 4, Loss: 0.503039
Epoch: 5, Loss: 0.484781
Epoch: 6, Loss: 0.492084
Epoch: 7, Loss: 0.510342
Epoch: 8, Loss: 0.459220
Epoch: 9, Loss: 0.543206
Epoch: 10, Loss: 0.473826
Epoch: 11, Loss: 0.459220
Epoch: 12, Loss: 0.565115
Epoch: 13, Loss: 0.528600
Epoch: 14, Loss: 0.499387
Epoch: 15, Loss: 0.535903
Epoch: 16, Loss: 0.488433
Epoch: 17, Loss: 0.470175


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in valloader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f", correct / total)

tensor([[-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        ...,
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028]])
tensor([[-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        ...,
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028]])
tensor([[-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        ...,
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028]])
tensor([[-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
        [-0.9960, -0.0028],
      